In [1]:
import pandas as pd
import numpy as np

from FRA import *

In [2]:
def find_no_ws(score, df):
    df1 = df.query('Score==@score')[['Category', 'Score', 'RegionGo']]
    df1 = df1.query('Category!="RS"')
    df1['No_WS'] = df1['Category'] != df1['RegionGo']
    print(df1['No_WS'].value_counts())
    print("WS:", str(np.round(df1['No_WS'].value_counts()[0]/df1['No_WS'].value_counts().sum()*100, 2)) + "%")
    return df1
'''
def find_no_ws(score, df):
    df1 = df.query('Score_LAG1==@score')[['Category_LAG1', 'Score_LAG1', 'Category']]
    df1 = df1.query('Category_LAG1!="RS"')
    df1['No_WS'] = df1['Category_LAG1'] != df1['Category']
    print(df1['No_WS'].value_counts())
    print("WS:", str(np.round(df1['No_WS'].value_counts()[0]/df1['No_WS'].value_counts().sum()*100, 2)) + "%")
    return df1
'''

'\ndef find_no_ws(score, df):\n    df1 = df.query(\'Score_LAG1==@score\')[[\'Category_LAG1\', \'Score_LAG1\', \'Category\']]\n    df1 = df1.query(\'Category_LAG1!="RS"\')\n    df1[\'No_WS\'] = df1[\'Category_LAG1\'] != df1[\'Category\']\n    print(df1[\'No_WS\'].value_counts())\n    print("WS:", str(np.round(df1[\'No_WS\'].value_counts()[0]/df1[\'No_WS\'].value_counts().sum()*100, 2)) + "%")\n    return df1\n'

In [13]:
df = pd.read_csv('../Data/humans_only_absent.csv')
# df2 = pd.read_csv('../Data/high_performing_human_dyads.csv')
df2 = pd.read_csv('../Data/humans_only_absent_perfect.csv')

In [4]:
for s in [29, 30, 31, 32]:
    print("************")
    print("Score:", s)
    find_no_ws(s, df)

************
Score: 29
False    15
True      5
Name: No_WS, dtype: int64
WS: 75.0%
************
Score: 30
False    20
True      7
Name: No_WS, dtype: int64
WS: 74.07%
************
Score: 31
False    65
True      4
Name: No_WS, dtype: int64
WS: 94.2%
************
Score: 32
False    397
True      12
Name: No_WS, dtype: int64
WS: 97.07%


<ipython-input-2-6c24b2e4dee1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['No_WS'] = df1['Category'] != df1['RegionGo']


In [14]:
for s in [29, 30, 31, 32]:
    print("************")
    print("Score:", s)
    find_no_ws(s, df2)

************
Score: 29
False    15
True      5
Name: No_WS, dtype: int64
WS: 75.0%
************
Score: 30
False    20
True      7
Name: No_WS, dtype: int64
WS: 74.07%
************
Score: 31
False    65
Name: No_WS, dtype: int64
WS: 100.0%
************
Score: 32
False    397
Name: No_WS, dtype: int64
WS: 100.0%


<ipython-input-2-6c24b2e4dee1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['No_WS'] = df1['Category'] != df1['RegionGo']


In [10]:
df3 = df.query('(Score==31)|(Score==32)')
df3 = df3.query('Category!="RS"')
df3['No_WS'] = df3['Category'] != df3['RegionGo']
df3 = df3[df3['No_WS']]
print(df3.shape)
df4 = df3[['Score', 'Category', 'Similarity', 'RegionGo',]]
df4

(16, 162)


,Score,Category,Similarity,RegionGo
90,31.0,ALL,1.000000,RS
164,31.0,NOTHING,0.031250,RS
277,32.0,NOTHING,1.000000,RS
312,32.0,ALL,1.000000,NOTHING
363,32.0,BOTTOM,0.968750,RS
367,32.0,BOTTOM,1.000000,RS
370,32.0,NOTHING,1.000000,RS
660,32.0,NOTHING,0.031250,RS
703,32.0,NOTHING,1.000000,BOTTOM
821,31.0,TOP,0.969697,RS


In [12]:
mask_false = df4.index
mask_true = [x for x in range(df.shape[0]) if x not in mask_false]
df5 = df.iloc[mask_true]
df5.to_csv('humans_only_absent_perfect.csv', index=False)

In [6]:
def crea_region(r, cols):
    
    lista = []
    for c in cols:
        lista.append(r[c])
    
    return lista

In [7]:
cols = ['a' + str(i+1) + str(j+1) for i in range(8) for j in range(8)]
df3['region'] = df3.apply(lambda x: crea_region(x, cols), axis=1)

In [11]:
regiones, estrategias = create_regions_and_strategies(8)

In [89]:
TOLERANCIA = 0.9

def classify_region(r, regiones, TOLERANCIA):
    
    # Check if close to NOTHING
    if np.array(r).sum() < 3:
        return('NOTHING')
    
    sims = [0]*8
    for i, k in enumerate(regiones):
        sims[i] = sim_consist(r, k)
        
    # sims_print = ["%.3f" % v for v in sims]
    # print('similarities:\n', sims)

    valor = np.max(np.array(sims))
    indiceMax = np.argmax(np.array(sims))
    print('valor:', valor, 'vale?:', valor >= TOLERANCIA)

    if valor >= TOLERANCIA:
        reg = nameRegion(indiceMax + 1)
        return(reg)
    else:
        return('RS')


In [6]:
df3 = df2.query('Score==32')
df3 = df3.query('Category!="RS"')
df3['No_WS'] = df3['Category'] != df3['RegionGo']
df3 = df3[df3['No_WS']]
print(df3.shape)
df3[['Score', 'Category', 'Similarity', 'RegionGo',]]

(9, 162)


,Score,Category,Similarity,RegionGo
113,32.0,NOTHING,1.00000,RS
148,32.0,ALL,1.00000,NOTHING
199,32.0,BOTTOM,0.96875,RS
203,32.0,BOTTOM,1.00000,RS
206,32.0,NOTHING,1.00000,RS
419,32.0,NOTHING,1.00000,BOTTOM
495,32.0,TOP,1.00000,RS
593,32.0,NOTHING,1.00000,RS
660,32.0,ALL,1.00000,RS


In [11]:
df3 = df2.query('Score_LAG1==32')
df3 = df3.query('Category_LAG1!="RS"')
df3['No_WS'] = df3['Category'] != df3['Category_LAG1']
df3 = df3[df3['No_WS']]
print(df3.shape)
df3[['Score_LAG1', 'Category_LAG1', 'Similarity_LAG1', 'Category', 'Similarity']]

(8, 162)


,Score_LAG1,Category_LAG1,Similarity_LAG1,Category,Similarity
114,32.0,NOTHING,1.000000,RS,0.392857
200,32.0,BOTTOM,0.968750,RS,0.812500
321,32.0,NOTHING,0.035714,RS,0.289474
335,32.0,NOTHING,1.000000,ALL,1.000000
417,32.0,NOTHING,1.000000,ALL,1.000000
496,32.0,TOP,1.000000,RS,0.888889
594,32.0,NOTHING,1.000000,RS,0.565217
611,32.0,TOP,1.000000,RS,0.562500


In [90]:
region = df3['region'].iloc[10]
imprime_region(region)
print("")
imprime_region(regiones[4])
sim_consist(region, regiones[4])

[1, 1, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]

[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]


0.90625

In [91]:
classify_region(region, regiones, TOLERANCIA)
#maxSim2Focal(region, 8)

valor: 0.90625 vale?: True


'LEFT'